# About:

This notebook basically finetunes a "Robust Resnet" by using a fine tuning dataset of PGD perturbed images and its augmentations

# Installing Libraries


In [ ]:
!pip cache purge

Files removed: 2913


In [ ]:
# !pip install pytorch-lightning
# !pip install lightning
# !pip install lightning[extra]
!pip install datasets
!pip install -U "huggingface_hub[cli]"
# !pip install tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 4.9 MB

# 0. Importing libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from datasets import load_dataset
from torch.optim import lr_scheduler

import uuid
import random
from torchvision.models import resnet50, ResNet50_Weights
from torch import nn
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm
import torchvision.transforms as transforms
import torch
import torch.nn.functional as F
from torchvision.transforms import Compose
from torchvision.transforms import ToTensor
import copy
import shutil
import zipfile
from torch.optim import lr_scheduler

# 1.Setup

In [ ]:
# Manual seed for reproducibility
# SEED = 1234
# torch.manual_seed(SEED)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"device: {DEVICE}")

base_path = "/content/drive/MyDrive/trustworthyml"
zips_path = os.path.join(base_path, "zips")
model_path = os.path.join(base_path, "model", "ft_stream")
logs_path = os.path.join(base_path, "logs")

device: cuda


# 2. Creating data loaders

In [ ]:
class ResnetPGDAttacker:
    def __init__(self):
        '''
        The PGD attack on Resnet model.
        :param model: The resnet model on which we perform the attack
        :param dataloader: The dataloader loading the input data on which we perform the attack
        '''
        self.resnet = resnet50(weights=ResNet50_Weights.DEFAULT)
        self.loss_fn = nn.CrossEntropyLoss()
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.resnet.to(self.device)

        # Nullify gradient for model params
        # for p in self.resnet.parameters():
        #     p.requires_grad = False

    def pgd_attack(self, images, labels, eps, alpha, steps):
        images = images.requires_grad_(True).to(self.device)
        labels = labels.to(self.device)
        adv_images = images.clone().requires_grad_(True).to(self.device)

        # Starting at a uniformly random point within the eps ball
        random_noise = torch.zeros_like(adv_images).uniform_(-eps, eps)
        adv_images = adv_images + random_noise

        for _ in range(steps):
            outputs = self.resnet(adv_images)
            pgd_loss = nn.CrossEntropyLoss()(outputs, labels)
            grad = torch.autograd.grad(pgd_loss, adv_images, retain_graph=True)[0]

            with torch.no_grad():
                adv_images = adv_images + alpha * grad.sign()
                adv_images = torch.clamp(adv_images, images - eps, images + eps)

            adv_images = adv_images.detach().requires_grad_(True).to(self.device)

        return adv_images

In [ ]:
class RobustDataLoader:
    def __init__(self, batch_num, batch_size, allow_perturbations=True, perturbation_chance=1.0):
        self.batch_num = batch_num
        self.batch_size = batch_size
        self.allow_perturbations = allow_perturbations
        self.pgd_attacker = ResnetPGDAttacker()
        self.perturbation_chance = perturbation_chance

        # Load the dataset
        self.ds = load_dataset("ILSVRC/imagenet-1k", split="train", streaming=True, trust_remote_code=True)
        self.ds = self.ds.shuffle()
        self.ds = self.ds.filter(lambda example: example['image'].mode == 'RGB')
        self.ds = self.ds.take(self.batch_num * self.batch_size)  # Take a fixed number of examples
        self.ds = self.ds.map(self.preprocess_img)

        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        # Create a DataLoader
        self.data_loader = DataLoader(self.ds, batch_size=self.batch_size)
        self.data_iter = iter(self.data_loader)

        print(
            f"Robust DataLoader created with batch_num = {self.batch_num}, batch_size = {self.batch_size}, perturbation_chance = {self.perturbation_chance}"
        )

    def preprocess_img(self, example):
        weights = ResNet50_Weights.DEFAULT
        transform = weights.transforms()
        example['image'] = transform(example['image'])
        return example

    def __iter__(self):
        return self

    def __next__(self):
        # Get the next batch from the DataLoader iterator
        try:
            batch = next(self.data_iter)
        except StopIteration:
            raise StopIteration  # Raise StopIteration when the DataLoader is exhausted

        # Get the next batch from the DataLoader
        images = batch['image']
        labels = batch['label']

        # Randomly apply PGD attack if allowed
        if self.allow_perturbations and random.random() < self.perturbation_chance:
            random_eps = random.uniform(0.01, 0.3)
            random_alpha = random.uniform(0.01, 0.07)
            random_steps = random.randint(15, 20)
            perturbed_images = self.pgd_attacker.pgd_attack(images, labels, random_eps, random_alpha, random_steps)
            return perturbed_images.to(self.device), labels.to(self.device)
        else:
            return images.to(self.device), labels.to(self.device)

    def __len__(self):
        return self.batch_num


# 3. Implementing Robust Resnet

In [ ]:
class RobustResnet:
    def __init__(self, data_loader, epochs, train_steps, val_steps, test_steps, checkpoint_dir, learning_rate=1e-3,
                 freeze_interval=10):
        self.model = resnet50(weights=ResNet50_Weights.DEFAULT)
        self.freeze_interval = freeze_interval
        self.loss_fn = nn.CrossEntropyLoss()
        self.data_loader = data_loader
        self.data_iter = iter(self.data_loader)
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.train_steps = train_steps
        self.val_steps = val_steps
        self.test_steps = test_steps
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        self.global_step = 0  # Track the number of training steps
        self.best_val_accuracy = 0.0  # Track the best validation accuracy
        self.checkpoint_dir = checkpoint_dir
        self.info = dict()

        # Learning rate scheduler (example: StepLR)
        self.scheduler = lr_scheduler.StepLR(self.optimizer, step_size=5, gamma=0.1)

    def load_model_from_checkpoint(self, checkpoint_path):
        checkpoint = torch.load(checkpoint_path)

        self.model.load_state_dict(checkpoint['state_dict'])
        self.model.to(self.device)
        print(f"Loaded model from path {checkpoint_path}")

    def training_phase(self):
        self.model.train()
        train_loss, train_correct, train_total = 0.0, 0, 0

        # Use TQDM for training
        pbar = tqdm(total=self.train_steps, desc="Training", colour="green")
        for _ in range(self.train_steps):
            try:
                batch = next(self.data_iter)
            except StopIteration:
                print("Training dataloader exhausted!")
                raise StopIteration

            if self.global_step % self.freeze_interval == 0:
                self.random_freeze_layers()

            images, labels = batch
            images, labels = images.to(self.device), labels.to(self.device)
            self.optimizer.zero_grad()
            outputs = self.model(images)
            loss = self.loss_fn(outputs, labels)
            loss.backward()
            self.optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
            self.global_step += 1

            # Update TQDM
            pbar.set_postfix(loss=loss.item(), accuracy=(predicted == labels).sum().item() / len(labels))
            pbar.update(1)

        pbar.close()

        # Step the scheduler at the end of the epoch
        self.scheduler.step()

        if self.train_steps == 0:
            return 0, 0

        epoch_train_loss = train_loss / self.train_steps
        epoch_train_accuracy = train_correct / train_total
        return epoch_train_loss, epoch_train_accuracy

    def validation_phase(self):
        self.model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0

        pbar = tqdm(total=self.val_steps, desc="Validation", colour="orange")
        for _ in range(self.val_steps):
            try:
                batch = next(self.data_iter)
            except StopIteration:
                print("Training dataloader exhausted!")
                raise StopIteration

            with torch.no_grad():
                images, labels = batch
                images, labels = images.to(self.device), labels.to(self.device)
                outputs = self.model(images)
                loss = self.loss_fn(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

                # Update TQDM
                pbar.set_postfix(loss=loss.item(), accuracy=(predicted == labels).sum().item() / len(labels))
                pbar.update(1)

        pbar.close()

        epoch_val_loss = val_loss / self.val_steps
        epoch_val_accuracy = val_correct / val_total
        return epoch_val_loss, epoch_val_accuracy

    def train(self):
        for epoch in range(1, self.epochs + 1):
            print("-" * 50)
            print(f"EPOCH {epoch}/{self.epochs}")

            epoch_train_loss, epoch_train_accuracy = self.training_phase()
            print(f"Epoch {epoch} Training Loss: {epoch_train_loss}, Accuracy: {epoch_train_accuracy}")

            epoch_val_loss, epoch_val_accuracy = self.validation_phase()
            print(f"Epoch {epoch} Validation Loss: {epoch_val_loss}, Accuracy: {epoch_val_accuracy}")

            self.info[f"epoch-{epoch}"] = {
                "train_loss": epoch_train_loss,
                "train_accuracy": epoch_train_accuracy,
                "val_loss": epoch_val_loss,
                "val_accuracy": epoch_val_accuracy,
                "is_checkpointed": False
            }

            if epoch_val_accuracy >= self.best_val_accuracy:
                print(f"SAVING NEW WEIGHTS!!")
                print()
                print(f"Previous best accuracy: {self.best_val_accuracy}, Current epoch accuracy: {epoch_val_accuracy}")
                self.best_val_accuracy = epoch_val_accuracy
                checkpoint_file_name = f"ft-stream-crazy-epoch-{epoch}-{str(uuid.uuid4())}.ckpt"
                checkpoint_path = os.path.join(self.checkpoint_dir, checkpoint_file_name)
                torch.save({"state_dict": self.model.state_dict()}, checkpoint_path)
                print(f"Checkpoint saved: {checkpoint_path}")
                self.info[f"epoch-{epoch}"]["is_checkpointed"] = True

        # saving the final state as is
        print("Going to save final checkpoint")
        checkpoint_file_name = f"ft-stream-crazy-epoch-final-{str(uuid.uuid4())}.ckpt"
        checkpoint_path = os.path.join(self.checkpoint_dir, checkpoint_file_name)
        torch.save({"state_dict": self.model.state_dict()}, checkpoint_path)
        print(f"Final Checkpoint saved: {checkpoint_path}")
        return self.info

    def test(self):
        self.model.eval()
        test_loss, test_correct, test_total = 0.0, 0, 0
        pbar = tqdm(total=self.test_steps, desc="Test", colour="red")
        for _ in range(self.test_steps):
            try:
                batch = next(self.data_iter)
            except StopIteration:
                print("Training dataloader exhausted!")
                raise StopIteration

            with torch.no_grad():
                images, labels = batch
                images, labels = images.to(self.device), labels.to(self.device)
                outputs = self.model(images)
                loss = self.loss_fn(outputs, labels)

                test_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                test_total += labels.size(0)
                test_correct += (predicted == labels).sum().item()

                # Update TQDM
                pbar.set_postfix(loss=loss.item(), accuracy=(predicted == labels).sum().item() / len(labels))
                pbar.update(1)

        pbar.close()

        epoch_test_loss = test_loss / self.test_steps
        epoch_test_accuracy = test_correct / test_total
        print(f"Test Loss: {epoch_test_loss}, Test Accuracy: {epoch_test_accuracy}")
        return epoch_test_loss, epoch_test_accuracy

    def random_freeze_layers(self):
        num_layers = len(list(self.model.children()))
        layers_to_freeze = int(num_layers * 0.6)

        # Unfreeze all layers first
        for layer in self.model.children():
            for param in layer.parameters():
                param.requires_grad = True

        # Randomly select layers to freeze
        layers_to_freeze_indices = random.sample(range(num_layers), layers_to_freeze)
        for i, child in enumerate(self.model.children()):
            if i in layers_to_freeze_indices:
                for param in child.parameters():
                    param.requires_grad = False  # Freeze parameters
            else:
                for param in child.parameters():
                    param.requires_grad = True

        # truly crazy approach , where we determine when to freeze next by again doing a random number pick
        self.freeze_interval = random.randint(20,50)


# 3. Training the model

In [ ]:
print("DONT RUN!")

In [ ]:
# buffer

In [ ]:
epochs = 15
train_steps = 1400
val_steps = 30
test_steps = 60
batch_size = 16
freeze_steps = 15

data_loader = RobustDataLoader(
    batch_num=epochs * (train_steps + val_steps) + test_steps,
    batch_size=batch_size,
    perturbation_chance=0.72
)

classifier = RobustResnet(data_loader, epochs=epochs, train_steps=train_steps, val_steps=val_steps,
                          test_steps=test_steps, checkpoint_dir=model_path)

classifier.train()



Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 192MB/s]


imagenet-1k.py:   0%|          | 0.00/4.58k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/85.4k [00:00<?, ?B/s]

classes.py:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

Robust DataLoader created with batch_num = 21510, batch_size = 16, perturbation_chance = 0.72
--------------------------------------------------
EPOCH 1/15


...

Epoch 14 Validation Loss: 2.92523060242335, Accuracy: 0.375
--------------------------------------------------
EPOCH 15/15


Training:   0%|          | 0/1400 [00:00<?, ?it/s]

Epoch 15 Training Loss: 2.6864608507922716, Accuracy: 0.41535714285714287


Validation:   0%|          | 0/30 [00:00<?, ?it/s]

Epoch 15 Validation Loss: 2.480913786093394, Accuracy: 0.45416666666666666
Previous best accuracy: 0.425, Current epoch accuracy: 0.45416666666666666
Checkpoint saved: /content/drive/MyDrive/trustworthyml/model/ft_stream/ft-stream-crazy-epoch-15.ckpt


In [ ]:
classifier.test()

Test:   0%|          | 0/60 [00:00<?, ?it/s]

Test Loss: 2.7773940642674764, Test Accuracy: 0.4114583333333333


(2.7773940642674764, 0.4114583333333333)

# 4. More training (from previous run)



In [ ]:
def get_newest_file(directory_path):
    # Initialize variables to hold the name of the most recent file and its modification time
    most_recent_file = None
    most_recent_time = 0

    # Iterate over the files in the directory using os.scandir()
    for entry in os.scandir(directory_path):
        if entry.is_file():  # Check if the entry is a file
            # Get the modification time of the file
            mod_time = entry.stat().st_mtime
            # Update if this file is more recent than the current most recent
            if mod_time >= most_recent_time:
                most_recent_file = entry.name  # Store the full path
                most_recent_time = mod_time

    return os.path.join(directory_path, most_recent_file)

In [ ]:
checkpoint_to_continue_training_from_path = get_newest_file(model_path)
print(checkpoint_to_continue_training_from_path)

/content/drive/MyDrive/trustworthyml/model/ft_stream/ft-stream-crazy-epoch-final-6fc9177b-fe57-4950-98f8-45ef577d1bda.ckpt


In [20]:
epochs = 35
train_steps = 1500
val_steps = 35
test_steps = 60
batch_size = 16


data_loader = RobustDataLoader(
    batch_num=epochs * (train_steps + val_steps) + test_steps,
    batch_size=batch_size,
    perturbation_chance=0.6 # we would like clean data as well
)

re_classifier = RobustResnet(data_loader, epochs=epochs, train_steps=train_steps, val_steps=val_steps,
                          test_steps=test_steps, checkpoint_dir=model_path)


# re_classifier.load_model_from_checkpoint(checkpoint_to_continue_training_from_path)

re_classifier.train()



Robust DataLoader created with batch_num = 53785, batch_size = 16, perturbation_chance = 0.6
--------------------------------------------------
EPOCH 1/35


Training:   0%|          | 0/1500 [00:00<?, ?it/s]

Epoch 1 Training Loss: 3.9863444365660348, Accuracy: 0.2425


Validation:   0%|          | 0/35 [00:00<?, ?it/s]

Epoch 1 Validation Loss: 3.9069802011762347, Accuracy: 0.23214285714285715
SAVING NEW WEIGHTS!!

Previous best accuracy: 0.0, Current epoch accuracy: 0.23214285714285715
Checkpoint saved: /content/drive/MyDrive/trustworthyml/model/ft_stream/ft-stream-crazy-epoch-1-c3c4b227-ffc6-462e-a9cc-e9d349df602d.ckpt
--------------------------------------------------
EPOCH 2/35


...

Training:   0%|          | 0/1500 [00:00<?, ?it/s]

Epoch 35 Training Loss: 2.4869305351177853, Accuracy: 0.4469166666666667


Validation:   0%|          | 0/35 [00:00<?, ?it/s]

Epoch 35 Validation Loss: 2.4348430923053197, Accuracy: 0.4660714285714286
Going to save final checkpoint
Final Checkpoint saved: /content/drive/MyDrive/trustworthyml/model/ft_stream/ft-stream-crazy-epoch-final-fc4d6b21-4fcc-4220-90c4-bf93a71a4068.ckpt


{'epoch-1': {'train_loss': 3.9863444365660348,
  'train_accuracy': 0.2425,
  'val_loss': 3.9069802011762347,
  'val_accuracy': 0.23214285714285715,
  'is_checkpointed': True},
 'epoch-2': {'train_loss': 3.8473707874615988,
  'train_accuracy': 0.242125,
  'val_loss': 3.8532280649457658,
  'val_accuracy': 0.21428571428571427,
  'is_checkpointed': False},
 'epoch-3': {'train_loss': 3.761446208000183,
  'train_accuracy': 0.24316666666666667,
  'val_loss': 3.6212562424795967,
  'val_accuracy': 0.26964285714285713,
  'is_checkpointed': True},
 'epoch-4': {'train_loss': 3.6292912650903064,
  'train_accuracy': 0.2612083333333333,
  'val_loss': 3.375184828894479,
  'val_accuracy': 0.3142857142857143,
  'is_checkpointed': True},
 'epoch-5': {'train_loss': 3.5420033915837608,
  'train_accuracy': 0.27316666666666667,
  'val_loss': 3.831254277910505,
  'val_accuracy': 0.2732142857142857,
  'is_checkpointed': False},
 'epoch-6': {'train_loss': 3.114917760372162,
  'train_accuracy': 0.341208333333333

In [21]:
re_classifier.test()

Test:   0%|          | 0/60 [00:00<?, ?it/s]

Test Loss: 2.2602140804131827, Test Accuracy: 0.49270833333333336


(2.2602140804131827, 0.49270833333333336)

# 5. Evaluating Results

In [ ]:
class Evaluation:
    def __init__(self, checkpoint_path, test_loader, evaluate_original=True, evaluate_finetuned=True):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.evaluate_original = evaluate_original
        self.evaluate_finetuned = evaluate_finetuned
        self.checkpoint_path = checkpoint_path
        self.test_loader = test_loader

        if self.evaluate_original:
            self.original_model = resnet50(weights=ResNet50_Weights.DEFAULT)
            self.original_model.to(self.device)
            self.original_model.eval()

        if self.evaluate_finetuned:
            self.fine_tuned_model = resnet50()
            self.load_model_from_checkpoint()
            self.fine_tuned_model.eval()

        print(f"evaluate original : {evaluate_original}")
        print(f"evaluate finetuned : {evaluate_finetuned}")

    def load_model_from_checkpoint(self):
        checkpoint = torch.load(self.checkpoint_path)

        self.fine_tuned_model.load_state_dict(checkpoint['state_dict'])
        self.fine_tuned_model.to(self.device)
        self.fine_tuned_model.eval()
        print(f"Loaded model from {self.checkpoint_path}")

    def evaluate_model(self):
        # Initialize accuracy metrics
        total = 0
        orig_correct = 0
        ft_correct = 0

        # Evaluate the original model
        for batch in tqdm(self.test_loader):
            images, labels = batch
            total += len(labels)

            images, labels = images.to(self.device), labels.to(self.device)

            # Original model predictions
            if self.evaluate_original:
                original_logits = self.original_model(images).softmax(1)
                original_predictions = original_logits.argmax(dim=1)
                orig_correct += torch.sum(original_predictions == labels).item()

            if self.evaluate_finetuned:
                ft_images = images.clone().detach().to(self.device)
                ft_labels = labels.clone().detach().to(self.device)

                # Fine-tuned model predictions
                fine_tuned_logits = self.fine_tuned_model(ft_images).softmax(1)
                fine_tuned_predictions = fine_tuned_logits.argmax(dim=1)
                ft_correct += torch.sum(fine_tuned_predictions == ft_labels).item()

        # Calculate accuracies
        result = {}
        if self.evaluate_original:
            original_accuracy = orig_correct / total
            print(f'Evaluation Original Model Accuracy: {original_accuracy * 100} %')
            result["original_accuracy"] = original_accuracy

        if self.evaluate_finetuned:
            fine_tuned_accuracy = ft_correct / total
            print(f'Evaluation Fine-Tuned Model Accuracy: {fine_tuned_accuracy * 100} %')
            result["fine_tuned_accuracy"] = fine_tuned_accuracy

        return result

In [23]:
clean_test_loader = RobustDataLoader(
    batch_num=65,
    batch_size=batch_size,
    perturbation_chance=0.0
)

perturbed_test_loader = RobustDataLoader(
    batch_num=65,
    batch_size=batch_size,
    perturbation_chance=1.0
)

mix_test_loader = RobustDataLoader(
    batch_num=65,
    batch_size=batch_size,
    perturbation_chance=0.55
)



Robust DataLoader created with batch_num = 65, batch_size = 16, perturbation_chance = 0.0
Robust DataLoader created with batch_num = 65, batch_size = 16, perturbation_chance = 1.0
Robust DataLoader created with batch_num = 65, batch_size = 16, perturbation_chance = 0.55


In [24]:
checkpoint_to_evaluate = get_newest_file(model_path)
print(checkpoint_to_evaluate)

/content/drive/MyDrive/trustworthyml/model/ft_stream/ft-stream-crazy-epoch-final-fc4d6b21-4fcc-4220-90c4-bf93a71a4068.ckpt


Testing on only clean data

In [25]:
evaluation = Evaluation(checkpoint_to_evaluate, clean_test_loader, evaluate_original=True, evaluate_finetuned=True)
evaluation.evaluate_model()

<ipython-input-22-f702b9de89eb>:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(self.checkpoint_path)


evaluate original : True
evaluate finetuned : True


  0%|          | 0/65 [00:00<?, ?it/s]

Evaluation Original Model Accuracy: 92.01923076923076 %
Evaluation Fine-Tuned Model Accuracy: 52.307692307692314 %


{'original_accuracy': 0.9201923076923076,
 'fine_tuned_accuracy': 0.5230769230769231}

Testing on only perturbed data

In [29]:
perturbed_test_loader = RobustDataLoader(
    batch_num=65,
    batch_size=batch_size,
    perturbation_chance=1.0
)


evaluation = Evaluation(checkpoint_to_evaluate, perturbed_test_loader, evaluate_original=True, evaluate_finetuned=True)
evaluation.evaluate_model()

Robust DataLoader created with batch_num = 65, batch_size = 16, perturbation_chance = 1.0


<ipython-input-22-f702b9de89eb>:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(self.checkpoint_path)


evaluate original : True
evaluate finetuned : True


  0%|          | 0/65 [00:00<?, ?it/s]

Evaluation Original Model Accuracy: 8.75 %
Evaluation Fine-Tuned Model Accuracy: 44.13461538461539 %


{'original_accuracy': 0.0875, 'fine_tuned_accuracy': 0.44134615384615383}

Testing on mix data

In [27]:
evaluation = Evaluation(checkpoint_to_evaluate, mix_test_loader, evaluate_original=True, evaluate_finetuned=True)
evaluation.evaluate_model()

<ipython-input-22-f702b9de89eb>:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(self.checkpoint_path)


evaluate original : True
evaluate finetuned : True


  0%|          | 0/65 [00:00<?, ?it/s]

Evaluation Original Model Accuracy: 52.78846153846154 %
Evaluation Fine-Tuned Model Accuracy: 49.80769230769231 %


{'original_accuracy': 0.5278846153846154,
 'fine_tuned_accuracy': 0.4980769230769231}

Testing another checkpoint which had highest validation accuracy

In [31]:
checkpoint_to_evaluate = "/content/drive/MyDrive/trustworthyml/model/ft_stream/ft-stream-crazy-epoch-31-1102d439-4c93-4a96-b1c6-a2ea21534227.ckpt"

In [32]:
clean_test_loader = RobustDataLoader(
    batch_num=65,
    batch_size=batch_size,
    perturbation_chance=0.0
)

perturbed_test_loader = RobustDataLoader(
    batch_num=65,
    batch_size=batch_size,
    perturbation_chance=1.0
)

mix_test_loader = RobustDataLoader(
    batch_num=65,
    batch_size=batch_size,
    perturbation_chance=0.55
)



Robust DataLoader created with batch_num = 65, batch_size = 16, perturbation_chance = 0.0
Robust DataLoader created with batch_num = 65, batch_size = 16, perturbation_chance = 1.0
Robust DataLoader created with batch_num = 65, batch_size = 16, perturbation_chance = 0.55


Clean data

In [33]:
evaluation = Evaluation(checkpoint_to_evaluate, clean_test_loader, evaluate_original=True, evaluate_finetuned=True)
evaluation.evaluate_model()

<ipython-input-22-f702b9de89eb>:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(self.checkpoint_path)


evaluate original : True
evaluate finetuned : True


  0%|          | 0/65 [00:00<?, ?it/s]

Evaluation Original Model Accuracy: 90.38461538461539 %
Evaluation Fine-Tuned Model Accuracy: 47.01923076923077 %


{'original_accuracy': 0.9038461538461539,
 'fine_tuned_accuracy': 0.4701923076923077}

Mix Data

In [34]:
evaluation = Evaluation(checkpoint_to_evaluate, mix_test_loader, evaluate_original=True, evaluate_finetuned=True)
evaluation.evaluate_model()

<ipython-input-22-f702b9de89eb>:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(self.checkpoint_path)


evaluate original : True
evaluate finetuned : True


  0%|          | 0/65 [00:00<?, ?it/s]

Evaluation Original Model Accuracy: 54.61538461538461 %
Evaluation Fine-Tuned Model Accuracy: 48.26923076923077 %


{'original_accuracy': 0.5461538461538461,
 'fine_tuned_accuracy': 0.4826923076923077}

Perturbed data

In [35]:
evaluation = Evaluation(checkpoint_to_evaluate, perturbed_test_loader, evaluate_original=True, evaluate_finetuned=True)
evaluation.evaluate_model()

<ipython-input-22-f702b9de89eb>:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(self.checkpoint_path)


evaluate original : True
evaluate finetuned : True


  0%|          | 0/65 [00:00<?, ?it/s]

Evaluation Original Model Accuracy: 9.615384615384617 %
Evaluation Fine-Tuned Model Accuracy: 45.76923076923077 %


{'original_accuracy': 0.09615384615384616,
 'fine_tuned_accuracy': 0.4576923076923077}